In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
print("Libraries imported")


Libraries imported


In [2]:
src = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(src, 'lxml')

table = soup.find('table', class_='wikitable sortable')

postal=[]
borough=[]
neigb=[]

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        if(cells[1].find(text=True).rstrip() != 'Not assigned'):
            postal.append(cells[0].find(text=True).rstrip())
            borough.append(cells[1].find(text=True).rstrip())
            if(cells[2].find(text=True).rstrip()!='Not assigned'):
                neigb.append(cells[2].find(text=True).rstrip())
            else:
                neigb.append(cells[1].find(text=True).rstrip())

print("table is ok")

table is ok


In [3]:
df = pd.DataFrame(postal, columns=['PostalCode'])
df['Borough']=borough
df['Neighborhood']=neigb
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [4]:
df_gp = df.groupby(['PostalCode','Borough'], sort=False).agg(lambda x: ','.join(x))
df_gp.reset_index(level=['PostalCode','Borough'], inplace=True)
df_gp.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
print('Total number of rows = {}'.format(df_gp.shape[0]))

Total number of rows = 103


In [6]:
df = df_gp

In [9]:
#!conda install -c conda-forge geocoder --yes
import geocoder
print('ok')

ok


In [12]:
lat = []
lng = []

for i in range(df.shape[0]):
    adr = '{} , Toronto, Ontario'.format(df.at[i, 'PostalCode'])
    lat_lng_cords = None
    while(lat_lng_cords is None):
        g = geocoder.arcgis(adr)
        lat_lng_cords= g.latlng
    lat.append(lat_lng_cords[0])
    lng.append(lat_lng_cords[1])
        

In [13]:
df['Latitude'] = lat
df['Longitude'] = lng

In [15]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [18]:
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("done")


done


In [20]:
address = 'Toronto, Ontario'

locator = Nominatim(user_agent='ny_explorer')
loc = locator.geocode(address)
lat = loc.latitude
lng = loc.longitude

43.653963


In [23]:
toronto_map = folium.Map(location=[lat, lng], zoom_start=10) 
for lat, lng, borough, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    lbl = '{} , {} '.format(neigh, borough)
    label = folium.Popup(lbl, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
toronto_map

In [26]:
# @hidden_cell
CLIENT_ID = 'ULJ50MWMNRKFLLWDI5CV05REIVRUVZFJMPO22NMFQFIRCD3S' # your Foursquare ID
CLIENT_SECRET = 'XDODSLUAVDPJQLJ4WA11BZONPS3Q5CJEDWKEVDM3AKRBKLNS' # your Foursquare Secret
VERSION = '20180605'

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            50)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [32]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752440,-79.329271,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752440,-79.329271,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.752440,-79.329271,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.730421,-79.313320,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.730421,-79.313320,Memories of Africa,43.726602,-79.312427,Grocery Store


In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Agincourt,15,15,15,15,15,15
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",1,1,1,1,1,1
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",14,14,14,14,14,14
"Alderwood,Long Branch",5,5,5,5,5,5
"Bathurst Manor,Downsview North,Wilson Heights",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,50,50,50,50,50,50


In [34]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_onehot.shape

(1833, 245)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.00,0.0,0.060000,0.000000,0.02,0.00,0.000000,0.04,...,0.000000,0.00000,0.00,0.020000,0.0,0.000000,0.000000,0.02,0.000000,0.000000
1,Agincourt,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.066667,0.00,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.071429,0.000000,0.00,0.000000,0.000000
4,"Alderwood,Long Branch",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
6,Bayview Village,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.25000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
8,Berczy Park,0.000000,0.00,0.0,0.000000,0.000000,0.02,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.020000,0.0,0.000000,0.000000,0.00,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.000000,0.00000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000


In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2  American Restaurant  0.06
3           Steakhouse  0.06
4                Hotel  0.06


----Agincourt----
                venue  freq
0         Supermarket  0.13
1       Shopping Mall  0.13
2  Chinese Restaurant  0.13
3     Bubble Tea Shop  0.07
4                Park  0.07


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                  Pharmacy   1.0
1                    Market   0.0
2            Massage Studio   0.0
3            Mattress Store   0.0
4  Mediterranean Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.14
1                 Park  0.07
2       Hardware Store  0.07
3          Coffee Shop  0.07
4  Japanese Restaurant  0.07


----Alderwood,Long Branch----
               venue

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Hotel,American Restaurant,Steakhouse,Asian Restaurant,Gastropub,Restaurant,Seafood Restaurant,Bar
1,Agincourt,Supermarket,Chinese Restaurant,Shopping Mall,Bakery,Badminton Court,Shanghai Restaurant,Sushi Restaurant,Pool,Bubble Tea Shop,Vietnamese Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Liquor Store,Japanese Restaurant,Beer Store,Coffee Shop,Sandwich Place,Pizza Place,Fried Chicken Joint,Pharmacy
4,"Alderwood,Long Branch",Gym,Convenience Store,Sandwich Place,Pub,Dance Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Women's Store
